In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col 
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

spark

df = spark.read.option("header", "true").csv("/home/iceberg/data/events.csv").withColumn("event_date", expr("DATE_TRUNC('day', event_time)"))

df.show()

+-----------+----------+--------+--------------------+----------+--------------------+-------------------+
|    user_id| device_id|referrer|                host|       url|          event_time|         event_date|
+-----------+----------+--------+--------------------+----------+--------------------+-------------------+
| 1037710827| 532630305|    NULL| www.zachwilson.tech|         /|2021-03-08 17:27:...|2021-03-08 00:00:00|
|  925588856| 532630305|    NULL|    www.eczachly.com|         /|2021-05-10 11:26:...|2021-05-10 00:00:00|
|-1180485268| 532630305|    NULL|admin.zachwilson....|         /|2021-02-17 16:19:...|2021-02-17 00:00:00|
|-1044833855| 532630305|    NULL| www.zachwilson.tech|         /|2021-09-24 15:53:...|2021-09-24 00:00:00|
|  747494706| 532630305|    NULL| www.zachwilson.tech|         /|2021-09-26 16:03:...|2021-09-26 00:00:00|
|  747494706| 532630305|    NULL|admin.zachwilson....|         /|2021-02-21 16:08:...|2021-02-21 00:00:00|
| -824540328| 532630305|    NULL|admi

In [36]:
sorted = df.repartition(10, col("event_date")) \
        .sortWithinPartitions(col("event_date"), col("host"))\
        .withColumn("event_time", col("event_time").cast("timestamp")) 

sorted.show()
                              

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.p

Py4JError: functions does not exist in the JVM

In [19]:
%%sql

CREATE DATABASE IF NOT EXISTS bootcamp

++
||
++
++

In [20]:
%%sql

DROP TABLE IF EXISTS bootcamp.events

++
||
++
++

In [21]:
%%sql

CREATE TABLE IF NOT EXISTS bootcamp.events (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (years(event_date));


++
||
++
++

In [22]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.events_sorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (years(event_date));

++
||
++
++

In [23]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.events_unsorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg;

++
||
++
++

In [33]:

start_df = df.repartition(4, col("event_date")).withColumn("event_time", col("event_time").cast("timestamp")) \
    

first_sort_df = start_df.sortWithinPartitions(col("event_date"), col('referrer'), col("host"))

sorted = df.repartition(10, col("event_date")) \
        .sortWithinPartitions(col("event_date")) \
        .withColumn("event_time", col("event_time").cast("timestamp")) \

start_df.write.mode("overwrite").saveAsTable("bootcamp.events_unsorted")
first_sort_df.write.mode("overwrite").saveAsTable("bootcamp.events_sorted")

In [35]:
%%sql
    
select * from demo.bootcamp.events_sorted

user_id,device_id,referrer,host,url,event_time,event_date
-488618451,532630305,None,admin.zachwilson.tech,/,2021-01-12 10:22:17.016000,2021-01-12 00:00:00
-21136712,589185851,None,admin.zachwilson.tech,/,2021-01-12 18:49:28.425000,2021-01-12 00:00:00
-414920062,589185851,None,admin.zachwilson.tech,/,2021-01-12 18:54:30.995000,2021-01-12 00:00:00
-414920062,589185851,None,admin.zachwilson.tech,/,2021-01-12 19:56:56.809000,2021-01-12 00:00:00
-694958230,589185851,None,admin.zachwilson.tech,/,2021-01-12 20:08:15.964000,2021-01-12 00:00:00
2105351485,-290659081,None,www.eczachly.com,/,2021-01-12 04:44:23.791000,2021-01-12 00:00:00
-2116612468,-843023486,None,www.eczachly.com,/,2021-01-12 01:13:53.762000,2021-01-12 00:00:00
-505490192,1885594540,None,www.eczachly.com,/,2021-01-12 04:04:30.165000,2021-01-12 00:00:00
-2142732711,-290659081,None,www.eczachly.com,/,2021-01-12 05:14:02.344000,2021-01-12 00:00:00
2075040354,-290659081,None,www.eczachly.com,/,2021-01-12 07:51:23.359000,2021-01-12 00:00:00


In [34]:
%%sql

SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'sorted' 
FROM demo.bootcamp.events_sorted.files

UNION ALL
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'unsorted' 
FROM demo.bootcamp.events_unsorted.files





size,num_files,sorted
4942116,4,sorted
5050846,4,unsorted


In [90]:
%%sql
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files FROM demo.bootcamp.events.files;

size,num_files
3145713,5


In [ ]:
%%sql 
SELECT COUNT(1) FROM bootcamp.matches_bucketed.files

count(1)
3665
